In [101]:
import meshplot as mp
import numpy as np
import open3d as o3d
import pymesh as pm

from scipy.spatial.distance import cdist

In [102]:
def draw(*shapes):
    pm_shapes = [s for s in shapes if type(s) is pm.Mesh]
    o3d_shapes = [s for s in shapes if s not in pm_shapes]
    for shape in pm_shapes:
        mp.plot(shape.vertices, shape.faces)
    if o3d_shapes:
        o3d.visualization.draw_geometries(o3d_shapes)

In [103]:
mesh = o3d.io.read_triangle_mesh('data/dataset/YellowToy01/yellow_push_toy_1_70000.obj')
point_cloud = o3d.io.read_point_cloud('data/pointcloud_sampled/YellowToy01/inputs/non_deformed_2_correspondences_zoom_2048.ply')
draw(mesh, point_cloud)

In [104]:
mesh_points = np.asarray(mesh.vertices)
cloud_points = np.asarray(point_cloud.points)

# https://stackoverflow.com/questions/8317022/get-intersecting-rows-across-two-2d-numpy-arrays

nrows, ncols = cloud_points.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [cloud_points.dtype]}

common_points = np.intersect1d(cloud_points.view(dtype), mesh_points.view(dtype))

common_points = common_points.view(cloud_points.dtype).reshape(-1, ncols)
assert common_points.shape == cloud_points.shape

In [105]:
mesh_v_merge_map = np.zeros(len(mesh_points), dtype=int)

distances = cdist(mesh_points, cloud_points)
mesh_v_merge_map = distances.argmin(axis=1)
assert len(np.unique(mesh_v_merge_map, axis=0)) == len(cloud_points)

In [106]:
mesh_points = cloud_points[mesh_v_merge_map]
mesh.vertices = o3d.utility.Vector3dVector(mesh_points)
draw(mesh, point_cloud)

In [107]:
mesh = pm.form_mesh(
    np.asarray(mesh.vertices),
    np.asanyarray(mesh.triangles)
)

In [109]:
print(mesh.num_vertices)
mesh, _ = pm.remove_duplicated_vertices(mesh)
print(mesh.num_vertices)
assert mesh.num_vertices == len(cloud_points)
draw(mesh)

2048
2048


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.054057…

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_4801c174-0b79-417b-8e9a-875288ede388'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_7282bd1a-3116-43ab-b56d-08a7565a3f09'.